In [1]:
import pandas as pd
import csv
import numpy as np
import random
import io

In [2]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# loading training data and doing some setup
adjs_df = pd.read_csv('Adjectives.csv',  usecols=[1], header = 0, delimiter=",", quoting=csv.QUOTE_NONE, 
                       encoding='utf-8')
adjs_df.columns = ['word']


adjs = adjs_df.word.tolist()
adjs = [i.replace(" ", "") for i in adjs]
text = " ".join(adjs).lower().replace("\"", "")
print(text[:100], len(text))

chars = sorted(list(set(text)))
# print(chars)

# making dicts for encoding of characters
char_indices = dict((v, c) for c,v in enumerate(chars))
indices_char = dict((c, v) for c,v in enumerate(chars))

maxlen = 30
step = 3
sequences = []
next_char = []

for i in range(0, len(text) - maxlen, step):
  sequences.append(text[i : i + maxlen])
  next_char.append(text[i + maxlen])

x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)
# 
for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_char[i]]] = 1


In [4]:
model = keras.Sequential()

model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
# try adam optimizer aswell
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

model.save('adj_generate_model.h5')

In [5]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
epochs = 40
batch_size = 128

for epoch in range(epochs):
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print("Epoch: " + str(epoch))

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        generated = ""
        sentence = text[start_index : start_index + maxlen]

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char
        # print("Generated: ", generated)

        with open("generated_adjs.txt", "a") as txt:
          # tend to get a lot of repeats for some seeds, filter out when reading back the file
          txt.write(generated + " ")
        

In [ ]:
generated_words = open('generated_adjs.txt', 'r')
generated_words = generated_words.read()

words_list = list([i for i in set(generated_words.split(" ")) if len(i) > 0])

print(words_list)

In [ ]:
definitions_dict = {}

for word in words_list:
  definition = input("What does {} mean again?\n".format(word))
  if definition == "exit":
    break
  if definition == "skip":
    continue
  else:
    definitions_dict[word] = [definition]
    example = input("Right, and what is an example of {} in a sentence?\n".format(word))
    definitions_dict[word].append(example)

In [ ]:
# print(definitions_dict)

In [ ]:
# convert to full dictionary entry

# makes a fake phonetic version with direct substitution of letters 
def make_phonetic(word):
  letters_phon = "æɓçɖɘɸɡʜɪjkɭɰɲøpqʁstʊʋwχyʒ"
  letters_alph = "abcdefghijklmnopqrstuvwxyz"
  replaced = [letters_phon[letters_alph.index(i)] for i in word]
  return "/{}/".format("".join(replaced))

# puts everything together and appends entry to text file
def make_entries():
  for key, value in definitions_dict.items():
    entry = key + "\n adjective \n" + make_phonetic(key) + "\n" + value[0] + "\n" + "- {}".format(value[1])

    with open("new_dictionary.txt", "a") as txt:
            txt.write(entry + "\n\n")

In [ ]:
make_entries()